In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
words = open('names.txt', 'r').read().splitlines()

In [8]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
vocab_size

27

In [12]:
# dataset generation with splits

def build_dataset(words):
    block_size = 3
    X, Y = [], []
    for w in words:
        #print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
block_size = 3

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182437, 3]) torch.Size([182437])
torch.Size([22781, 3]) torch.Size([22781])
torch.Size([22928, 3]) torch.Size([22928])


In [13]:
# utility function we will use later when comparing manual gradients to PyTorch gradients 
def cmp(s, dt, t): 
    ex = torch.all(dt = t.grad).item() 
    app = torch.allclosed(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item() 
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [14]:
n_embd = 10
n_hidden = 64


g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd),              generator = g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden),  generator = g) * ((5/3) / ((n_embd * block_size)**0.5)) # multiply by gain/sqrt(fan_in)
b1 = torch.randn(n_hidden,                         generator = g) * 0.1 
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),           generator = g) * 0.1
b2 = torch.randn(vocab_size,                       generator = g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
for p in parameters:
    p.requires_grad = True

sum(p.nelement() for p in parameters)

4137

In [15]:
batch_size = 32
n = batch_size
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [ ]:
#foreward pass - chunkated into smaller steps

emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)

# Linear Layer 1
hprebn = embcat @ W1 + b1
# Batchnorm layer
bnmeani = 1/(n*hprebn.sum(0, keepdim=True))
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5)**0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-Linearity
h = torch.tanh(hpreact)
# Linear Layer 2
logits = h @ W2 + b2
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdem=True).values


